### 필요한 라이브러리

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import defaultdict
import csv
import re

### COSPI 200 편입 종목 크롤링

In [8]:
BaseUrl = 'http://finance.naver.com/sise/entryJongmok.nhn?&page='

# 종목 코드 추출
stock_codes = []

for i in range(1, 21):  # range() 함수의 끝 파라미터는 포함되지 않는다.
    url = BaseUrl + str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        txt = item.a.get('href')  # 예: https://finance.naver.com/item/main.nhn?code=006390
        k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            stock_codes.append(code)

print(stock_codes)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_73063/155905241.py:13: SyntaxWarning: invalid escape sequence '\d'
  k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복


['005930', '000660', '373220', '207940', '005380', '012450', '068270', '000270', '329180', '105560', '035420', '055550', '012330', '042660', '138040', '005490', '028260', '009540', '034020', '086790', '259960', '000810', '032830', '035720', '010130', '015760', '011200', '051910', '096770', '033780', '316140', '030200', '010140', '064350', '024110', '006400', '402340', '066570', '017670', '352820', '267260', '323410', '003550', '018260', '003670', '034730', '000100', '009150', '047050', '086280', '326030', '047810', '003490', '272210', '042700', '003230', '090430', '079550', '006800', '010620', '005830', '021240', '267250', '010120', '010950', '180640', '051900', '032640', '009830', '161390', '005940', '000150', '271560', '029780', '241560', '016360', '000720', '071050', '298040', '377300', '034220', '006260', '450080', '011790', '251270', '028050', '022100', '001040', '000880', '097950', '078930', '036460', '035250', '128940', '039490', '175330', '011070', '138930', '004020', '454910',

### 편입 종목을 같은 업종코드(산업군)끼리 분류 후 클러스터로 묶음

In [22]:
# 헤더 설정
headers = {"User-Agent": "Mozilla/5.0"}

# 종목코드로 업종코드를 추출하는 함수
def get_upjong_code_by_stock(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        upjong_link = soup.find("a", href=lambda href: href and "sise_group_detail.naver?type=upjong" in href)
        if upjong_link:
            href = upjong_link["href"]
            upjong_code = href.split("no=")[-1]
            return upjong_code
    except Exception as e:
        print(f"[오류] {stock_code} 처리 중 예외 발생: {e}")
    return None

# 종목코드로 종목명을 가져오는 함수
def get_stock_name_from_code(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        name = soup.select_one('div.wrap_company h2 a').text.strip()
        return name
    except:
        return None

# 업종코드별 종목 저장
upjong_groups = defaultdict(list)
stock_code_to_name = {}

for i, code in enumerate(stock_codes):
    name = get_stock_name_from_code(code)
    upjong_code = get_upjong_code_by_stock(code)
    if name:
        stock_code_to_name[code] = name
    if upjong_code:
        upjong_groups[upjong_code].append(code)
        print(f"[{i+1}/{len(stock_codes)}] {code}({name}) → 업종코드 {upjong_code}")
    else:
        print(f"[{i+1}/{len(stock_codes)}] {code}({name}) → 업종코드 없음")
    time.sleep(0.3)

# 업종코드별로 Cluster ID 부여
upjong_to_cluster = {upjong: idx for idx, upjong in enumerate(upjong_groups.keys())}

# 결과 정리
result = []
for upjong_code, codes in upjong_groups.items():
    cluster_id = upjong_to_cluster[upjong_code]
    for code in codes:
        result.append({
            "종목명": stock_code_to_name.get(code, code),
            "업종코드": upjong_code,
            "Cluster": cluster_id
        })

# 데이터프레임으로 변환
df = pd.DataFrame(result)
print("\n📦 최종 결과:")
print(df)

[1/200] 005930(삼성전자) → 업종코드 278
[2/200] 000660(SK하이닉스) → 업종코드 278
[3/200] 373220(LG에너지솔루션) → 업종코드 283
[4/200] 207940(삼성바이오로직스) → 업종코드 261
[5/200] 005380(현대차) → 업종코드 273
[6/200] 012450(한화에어로스페이스) → 업종코드 284
[7/200] 068270(셀트리온) → 업종코드 261
[8/200] 000270(기아) → 업종코드 273
[9/200] 329180(HD현대중공업) → 업종코드 291
[10/200] 105560(KB금융) → 업종코드 301
[11/200] 035420(NAVER) → 업종코드 300
[12/200] 055550(신한지주) → 업종코드 301
[13/200] 012330(현대모비스) → 업종코드 270
[14/200] 042660(한화오션) → 업종코드 291
[15/200] 138040(메리츠금융지주) → 업종코드 321
[16/200] 005490(POSCO홀딩스) → 업종코드 304
[17/200] 028260(삼성물산) → 업종코드 276
[18/200] 009540(HD한국조선해양) → 업종코드 291
[19/200] 034020(두산에너빌리티) → 업종코드 299
[20/200] 086790(하나금융지주) → 업종코드 301
[21/200] 259960(크래프톤) → 업종코드 263
[22/200] 000810(삼성화재) → 업종코드 315
[23/200] 032830(삼성생명) → 업종코드 330
[24/200] 035720(카카오) → 업종코드 300
[25/200] 010130(고려아연) → 업종코드 322
[26/200] 015760(한국전력) → 업종코드 325
[27/200] 011200(HMM) → 업종코드 323
[28/200] 051910(LG화학) → 업종코드 272
[29/200] 096770(SK이노베이션) → 업종코드 313
[30/200] 033780(KT

In [23]:
# 클러스터 개수와 종목 확인
df

,종목명,업종코드,Cluster
0,삼성전자,278,0
1,SK하이닉스,278,0
2,한미반도체,278,0
3,LG에너지솔루션,283,1
4,삼성SDI,283,1
...,...,...,...
195,에스디바이오센서,281,47
196,덴티움,281,47
197,한샘,303,48
198,동원시스템즈,311,49


### CSV로 저장

In [25]:
# 종목코드 앞에 0이오면 0이 사라진 후 저장되는 문제가 발생
# 그 이유는 종목코드를 int로 인지했기 때문. 그래서 문자열로 바꿔줌
# 000660 같은 경우, 660으로 저장되었었음

df.to_csv("종목_업종_클러스터.csv", index=False)

### 각 기업의 재무제표 데이터 합치기

In [13]:
df_cluster = pd.read_csv("/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/code/종목_업종_클러스터_test.csv")
df_group = pd.read_csv("/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/code/financial_data_processing_cospi200_test.csv")

# 병합 수행
df_merged = pd.merge(df_cluster, df_group, left_on='종목명', right_on='기업명_그룹', how='left')

# 병합용 열 제거
df_merged = df_merged.drop(columns=['기업명_그룹'])

# df_merged.to_csv("병합된_데이터프레임.csv", index=False)
# # 결과 확인
# tools.display_dataframe_to_user(name="병합된 데이터프레임", dataframe=df_merged)

### 각 클러스터 평가

### 두산로보틱스를 먼저 드랍해야함

In [14]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler

# 평가에 사용할 수치형 열만 선택 (Cluster 제외)
numeric_cols = df_merged.select_dtypes(include='number').columns.drop('Cluster')

# 1. 정규화 (스케일링)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_merged[numeric_cols])

# 2. 클러스터 라벨
labels = df_merged['Cluster']

# 3. 평가지표 계산
sil_score = silhouette_score(X_scaled, labels)
ch_score = calinski_harabasz_score(X_scaled, labels)
db_score = davies_bouldin_score(X_scaled, labels)

# 4. 결과 출력
print(f"✅ Silhouette Score: {sil_score:.4f}")
print(f"✅ Calinski-Harabasz Index: {ch_score:.4f}")
print(f"✅ Davies-Bouldin Index: {db_score:.4f}")

ValueError: Input X contains NaN.